<a href="https://colab.research.google.com/github/LuciaSS27/Proyecto-Datos/blob/main/Proyecto_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
## Hola
print("Hola mundo")

Hola mundo
